## Imports

In [1]:
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

import psycopg2
from psycopg2 import sql
import pandas as pd


/usr/local/python/3.12.1/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Look for knowledge base (feedback)

In [2]:
def get_db_connection():
    return psycopg2.connect(
        dbname="banksa",
        user="root",
        password="root",
        host="localhost",
        port="5432"
    )

In [4]:
def get_knowledge_base_not_indexed():
    cur = None
    conn = None
    try:
        conn = get_db_connection()
        cur = conn.cursor()

        query = sql.SQL("select * from v_knowledge_base")
        
        cur.execute(query)

        column_names = [desc[0] for desc in cur.description]
        results = cur.fetchall()

        df = pd.DataFrame(results, columns=column_names)

        return df
    except Exception as e:
        print(f"An error ocurred: {str(e)}")
        return pd.DataFrame()
    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()

In [6]:
df_knowledge_base = get_knowledge_base_not_indexed()
df_knowledge_base = df_knowledge_base[['id', 'description', 'category']]
df_knowledge_base.sample()

,id,description,category
42,76,BR FACTORY.COM 2624,ONLINE SHOPPING


In [7]:
model_name = 'all-MiniLM-L6-v2'
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
kb = df_knowledge_base.to_dict('records')
kb[0]

{'id': 87, 'description': 'FRALUSA SRL GRANIER', 'category': 'RESTAURANT'}

## Define evaluation functions 
1. **Hit Rate**
2. **MRR** Mean Reciprocal Rank

In [20]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)
    
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(knowledge_base, search_function):
    relevance_total = []

    for d in tqdm(knowledge_base):
        doc_id = d['id']
        results = search_function(d)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [11]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "description": {"type": "text"},
            "category": {"type": "text"},
            "description_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "category_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "description_category_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "categories_evaluation"

if es_client.indices.exists(index=index_name):
    print(f'Deleting index {index_name}') 
    es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'categories_evaluation'})

## Create Vector DB Index to store the embeddings for each document in the Knowledge Base

In [12]:
for doc in tqdm(kb):
    cat = doc['category']
    desc = doc['description']
    desc_cat = desc+ ' ' +cat

    doc['description_vector'] = embedding_model.encode(desc)
    doc['category_vector'] = embedding_model.encode(cat)
    doc['description_category_vector'] = embedding_model.encode(desc_cat)

  0%|          | 0/46 [00:00<?, ?it/s]

In [14]:
for doc in tqdm(kb):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/46 [00:00<?, ?it/s]

In [24]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 4,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["id", "description", "category"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

#### Define functions to vector search every field
1. Description
2. Category
3. Description + Cateory

In [22]:

def description_vector_knn(q):
    description = q['description']

    v_q = embedding_model.encode(description)

    return elastic_search_knn('description_vector', v_q)


def category_vector_knn(q):
    description = q['description']

    v_q = embedding_model.encode(description)

    return elastic_search_knn('category_vector', v_q)
    
def description_category_vector_knn(q):
    description = q['description']

    v_q = embedding_model.encode(description)

    return elastic_search_knn('description_category_vector', v_q)

## The results of every evaluation
1. Looks like a simple description (with vector) search can be quite handy

In [25]:
evaluate(kb, description_vector_knn)

  0%|          | 0/46 [00:00<?, ?it/s]

{'hit_rate': 1.0, 'mrr': 0.9565217391304348}

In [26]:
evaluate(kb, category_vector_knn)

  0%|          | 0/46 [00:00<?, ?it/s]

{'hit_rate': 0.2391304347826087, 'mrr': 0.161231884057971}

In [27]:
evaluate(kb, description_category_vector_knn)

  0%|          | 0/46 [00:00<?, ?it/s]

{'hit_rate': 1.0, 'mrr': 0.9565217391304348}